You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [30]:
import pandas as pd
# Start coding here... 

# a) Reading in the datasets 
# a.1) Reading in office address 
office_addresses_df = pd.read_csv('datasets/office_addresses.csv')
print("office address  DataFrame loaded successfully:") 
print("\n")

print(office_addresses_df.head())
print(office_addresses_df.columns)
print("\n")

# a.2) Reading in employee information 
employee_info_df = pd.read_excel('datasets/employee_information.xlsx')

print("employee information DataFrame loaded successfully:") 
print("\n")
print(employee_info_df.head())
print(employee_info_df.columns)
print("\n")

# a.3) Reading in emergency_contacts 

emergency_contacts_header = ["employee_id", "last_name", "first_name", "emergency_contact", "emergency_contact_number", "relationship"]

emergency_contacts_df = pd.read_excel('datasets/employee_information.xlsx', sheet_name=1, header=None, names=emergency_contacts_header)

print("employee emergency contact number DataFrame loaded successfully:") 
print("\n")
print(emergency_contacts_df.head())
print(emergency_contacts_df.columns) 
print("\n")

# a.4) Reading in employee roles. 

employee_roles = pd.read_json('datasets/employee_roles.json', orient="index") 
print("Employee Roles DataFrame loaded successfully:") 

print(employee_roles.head())
print("Employee Roles Columns:")
print(employee_roles.columns) 
print("\n")



office address  DataFrame loaded successfully:


          office office_country  ...   office_street office_street_number
0  Leuven Office             BE  ...  Martelarenlaan                   38
1     ESB Office             US  ...    Fifth Avenue                  350
2  WeWork Office             GB  ...      Old Street                  207

[3 rows x 5 columns]
Index(['office', 'office_country', 'office_city', 'office_street',
       'office_street_number'],
      dtype='object')


employee information DataFrame loaded successfully:


  employee_id employee_last_name  ...      employee_street employee_street_number
0      A2R5H9             Hunman  ...          Grote Markt                      9
1      H8K0L6               Siff  ...         Baker Street                    221
2      G4R7V0              Sagal  ...         Perry Street                     66
3      M1Z7U9             Coates  ...  Rue de l'Université                      7

[4 rows x 7 columns]
Index(['employee_id', 'e

In [31]:
# Merging the DataFrames

# Employee data merged with office addresses
employees = pd.merge(employee_info_df, office_addresses_df, left_on='employee_country', right_on='office_country', how='left')
print("Employee data merged with office addresses successfully (DataFrame 'employees'):")
print(employees.head())
print("\n")
print("Columns after merge:")
print(employees.columns)
print("\n")

# Merging employees with roles 

employees = pd.merge(employees, employee_roles, left_on='employee_id', right_index=True) 
print("Employee data merged with roles successfully (DataFrame 'employees'):") 
print(employees.head()) 
print("\n") 
print("Columns after roles merge:") 
print(employees.columns) 
print("\n")

# merging employees with emergency_contacts

employees = pd.merge(employees, emergency_contacts_df, on='employee_id') 
print("Employee data merged with emergency contacts successfully (DataFrame 'employees'):")
print(employees.head()) 
print("\n") 
print("Columns after emergency contacts merge:")
print(employees.columns) 
print("\n")


Employee data merged with office addresses successfully (DataFrame 'employees'):
  employee_id employee_last_name  ...   office_street office_street_number
0      A2R5H9             Hunman  ...  Martelarenlaan                 38.0
1      H8K0L6               Siff  ...      Old Street                207.0
2      G4R7V0              Sagal  ...    Fifth Avenue                350.0
3      M1Z7U9             Coates  ...             NaN                  NaN

[4 rows x 12 columns]


Columns after merge:
Index(['employee_id', 'employee_last_name', 'employee_first_name',
       'employee_country', 'employee_city', 'employee_street',
       'employee_street_number', 'office', 'office_country', 'office_city',
       'office_street', 'office_street_number'],
      dtype='object')


Employee data merged with roles successfully (DataFrame 'employees'):
  employee_id employee_last_name  ... monthly_salary               team
0      A2R5H9             Hunman  ...          $4500         Leadership
1    

In [32]:
# Cleaning and formatting the data
# Filling null values 

for col in employees.columns:
    if col.startswith('office'):
        employees[col].fillna("Remote", inplace=True)
        

print("Null values in 'office' columns filled with 'Remote':") 
print(employees.head()) 
print("\n") 
print("Checking for nulls in 'office_street' after fillna:") 
print(employees['office_street'].isnull().sum()) 
print("\n")

# subsetting and reindexing the dataframe 

final_columns = [ 'employee_id',
'employee_first_name', 'employee_last_name', 'employee_country', 'employee_city',
'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number',
'relationship', 'monthly_salary', 'team', 'title', 'office', 'office_country',
'office_city', 'office_street', 'office_street_number'
]

employees_final = employees[final_columns]
employees_final.set_index("employee_id", inplace=True)
print("DataFrame subsetted and reindexed successfully (DataFrame 'employees_final'):") 
print(employees_final.head()) 
print("\n") 
print("Columns of employees_final:") 
print(employees_final.columns) 
print("\n") 
print("Index of employees_final:") 
print(employees_final.index.name) 
print("\n")


Null values in 'office' columns filled with 'Remote':
  employee_id employee_last_name  ... emergency_contact_number relationship
0      A2R5H9             Hunman  ...          +32-456-5556-84      Brother
1      H8K0L6               Siff  ...         +44-020-5554-333       Sister
2      G4R7V0              Sagal  ...           +1-202-555-194      Husband
3      M1Z7U9             Coates  ...          +1-202-555-0130         Wife

[4 rows x 20 columns]


Checking for nulls in 'office_street' after fillna:
0


DataFrame subsetted and reindexed successfully (DataFrame 'employees_final'):
            employee_first_name  ... office_street_number
employee_id                      ...                     
A2R5H9                      Jax  ...                 38.0
H8K0L6                     Tara  ...                207.0
G4R7V0                    Gemma  ...                350.0
M1Z7U9                      Tig  ...               Remote

[4 rows x 17 columns]


Columns of employees_final:
Index(